# Parameter optimization

In [2]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['Pattern', 'LinearReg']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 1000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {'use_vol': [-1], 'window_low_bound': [1], 'window_high_bound': [6],
                            'first_candle': [0.5], 'second_candle': [0.5], 
                            'third_candle': [0.5]},
                'LinearReg': {'timeperiod': [8], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h, trade type is sell
Number of combinations is 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:26<00:00, 86.01s/it]


# Check local statistics

In [2]:
# 15m/1h
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']

pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_avg', ascending=False)
stat.head(10)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
99,Pattern_LinearReg,-1,1,6,0.9,0.9,0.5,8,0,175,1.001117,-0.387500,0.195417,-67.812500,1.1006,1.1488,1.1214,1.2247,1.2536,1.2348,1.2250,1.1355,1.0837,1.0473,1.0000,0.9787,0.9609,0.9303,0.9109,0.9090,0.8879,0.8644,0.8519,0.8750,0.8588,0.8330,0.7973,0.7933,-0.31,-0.41,-0.46,-0.49,-0.52,-0.65,-0.50,-0.53,-0.42,-0.43,-0.31,-0.43,-0.47,-0.52,-0.57,-0.55,-0.47,-0.30,-0.27,-0.28,-0.25,-0.08,0.0,-0.08
83,Pattern_LinearReg,-1,1,6,0.9,0.5,0.5,8,0,551,0.995313,-0.190000,-2.582812,-104.690000,1.4393,1.3846,1.3602,1.3101,1.2532,1.2421,1.1801,1.0873,1.0280,0.9919,0.9588,0.9335,0.9256,0.8920,0.8719,0.8524,0.8288,0.8073,0.7923,0.7828,0.7662,0.7533,0.7317,0.7141,-0.39,-0.45,-0.44,-0.41,-0.39,-0.45,-0.33,-0.37,-0.34,-0.36,-0.29,-0.20,-0.11,-0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
199,Pattern_LinearReg,0,1,6,0.9,0.9,0.5,8,0,230,0.989912,-0.415417,-2.320125,-95.545833,1.1196,1.1447,1.1672,1.2500,1.2407,1.1809,1.1672,1.0930,1.0658,1.0341,0.9842,0.9614,0.9413,0.9127,0.8960,0.9003,0.8692,0.8482,0.8389,0.8578,0.8484,0.8310,0.8045,0.8008,-0.38,-0.46,-0.48,-0.51,-0.52,-0.52,-0.45,-0.46,-0.41,-0.51,-0.32,-0.43,-0.49,-0.56,-0.57,-0.54,-0.49,-0.41,-0.27,-0.42,-0.41,-0.35,0.0,-0.01
91,Pattern_LinearReg,-1,1,6,0.9,0.7,0.5,8,0,371,0.988475,-0.180833,-4.275775,-67.089167,1.4044,1.4135,1.3240,1.3245,1.2872,1.2593,1.1825,1.0689,1.0188,0.9787,0.9435,0.9203,0.9078,0.8780,0.8537,0.8358,0.8113,0.7972,0.7787,0.7792,0.7692,0.7523,0.7255,0.7091,-0.38,-0.47,-0.44,-0.43,-0.40,-0.45,-0.33,-0.34,-0.32,-0.31,-0.24,-0.15,-0.05,-0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
15,Pattern_LinearReg,-1,1,6,0.5,0.8,0.5,8,0,836,0.981317,-0.293333,-15.619267,-245.226667,1.2555,1.2614,1.2473,1.2086,1.1783,1.1339,1.1025,1.0578,1.0316,1.0083,0.9621,0.9595,0.9327,0.9052,0.8704,0.8685,0.8642,0.8546,0.8339,0.8218,0.8182,0.7989,0.7950,0.7814,-0.37,-0.44,-0.47,-0.46,-0.46,-0.45,-0.36,-0.42,-0.43,-0.40,-0.42,-0.41,-0.31,-0.36,-0.31,-0.32,-0.20,-0.16,-0.18,-0.07,-0.04,0.00,0.0,0.00
19,Pattern_LinearReg,-1,1,6,0.5,0.9,0.5,8,0,523,0.974375,-0.289167,-13.401875,-151.234167,1.1570,1.1882,1.1644,1.1661,1.1457,1.1226,1.1165,1.0755,1.0331,1.0320,0.9698,0.9590,0.9242,0.8988,0.8753,0.8747,0.8672,0.8601,0.8384,0.8381,0.8343,0.8099,0.8217,0.8124,-0.33,-0.40,-0.43,-0.43,-0.47,-0.47,-0.39,-0.44,-0.44,-0.37,-0.34,-0.40,-0.29,-0.32,-0.31,-0.34,-0.22,-0.16,-0.19,-0.06,-0.08,-0.06,0.0,0.00
11,Pattern_LinearReg,-1,1,6,0.5,0.7,0.5,8,0,1135,0.972262,-0.257917,-31.482063,-292.735417,1.2730,1.3099,1.2391,1.2120,1.1789,1.1219,1.0793,1.0369,1.0051,0.9870,0.9578,0.9538,0.9273,0.8955,0.8664,0.8568,0.8521,0.8410,0.8084,0.8050,0.8005,0.7855,0.7772,0.7639,-0.37,-0.43,-0.43,-0.44,-0.44,-0.41,-0.32,-0.35,-0.38,-0.40,-0.39,-0.37,-0.30,-0.32,-0.31,-0.26,-0.12,-0.10,-0.05,0.00,0.00,0.00,0.0,0.00
119,Pattern_LinearReg,0,1,6,0.5,0.9,0.5,8,0,753,0.969329,-0.330417,-23.095137,-248.803750,1.1899,1.2176,1.1886,1.1960,1.1276,1.0814,1.0623,1.0283,0.9970,1.0103,0.9655,0.9512,0.9176,0.8991,0.8802,0.8804,0.8676,0.8574,0.8360,0.8344,0.8312,0.8122,0.8198,0.8123,-0.40,-0.46,-0.50,-0.50,-0.52,-0.47,-0.41,-0.4

# Research parameter influence

In [8]:
param = 'Pattern_second_candle'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_second_candle,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,0.5,0.854897,-0.304701,-168.318122,-286.847375,68130
1,0.6,0.850164,-0.288188,-150.008786,-258.243681,60370
2,0.7,0.849519,-0.281743,-133.779457,-224.315035,52007
3,0.8,0.839842,-0.299785,-109.302578,-205.237062,41617
4,0.9,0.871600,-0.322160,-72.804460,-170.303528,30776


# Plot result in file

In [11]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [12]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['Pattern', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'Pattern': {'window_low_bound': [2], 'window_high_bound': [5],
                            'first_candle': [0.6], 'second_candle': [0.8]},
                'LinearReg': {'timeperiod': [3], 'low_bound': [0]}
             }

        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)